In [15]:
# Dependencies
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [16]:
#Read csv file
disaster_df = pd.read_csv('./Resources/Climate-related_Disasters_Frequency.csv')

disaster_df.head()

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,...,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
0,1,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
1,2,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,4.0,2.0,1.0,4.0,1.0,3.0,6.0,5.0,2.0,5.0
3,4,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,1.0,NaN,4.0,NaN,2.0,1.0,1.0,1.0,1.0,1.0
4,5,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN


In [17]:
# Split off indicator from uneeded text
disaster_df['Indicator'] = disaster_df['Indicator'].str.split(':').str[1]

,Country,Indicator,F1980,F1981,F1982,F1983,F1984,F1985,F1986,F1987,...,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
0,"Afghanistan, Islamic Rep. of",Drought,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
1,"Afghanistan, Islamic Rep. of",Extreme temperature,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Afghanistan, Islamic Rep. of",Flood,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,2.0,1.0,4.0,1.0,3.0,6.0,5.0,2.0,5.0
3,"Afghanistan, Islamic Rep. of",Landslide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1.0,NaN,4.0,NaN,2.0,1.0,1.0,1.0,1.0,1.0
4,"Afghanistan, Islamic Rep. of",Storm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,Zambia,TOTAL,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,2.0,3.0,1.0,1.0
966,Zimbabwe,Drought,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
967,Zimbabwe,Flood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
968,Zimbabwe,Storm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,1.0,1.0,NaN,1.0,NaN,1.0,1.0


In [18]:
# Select columns for graphs
disaster_df = pd.concat([disaster_df['Country'], disaster_df['Indicator'],disaster_df.loc[:, 'F1980':'F2022']], axis=1)

disaster_df

,Country,Indicator,F1980,F1981,F1982,F1983,F1984,F1985,F1986,F1987,...,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
0,"Afghanistan, Islamic Rep. of",Drought,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
1,"Afghanistan, Islamic Rep. of",Extreme temperature,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Afghanistan, Islamic Rep. of",Flood,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,2.0,1.0,4.0,1.0,3.0,6.0,5.0,2.0,5.0
3,"Afghanistan, Islamic Rep. of",Landslide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1.0,NaN,4.0,NaN,2.0,1.0,1.0,1.0,1.0,1.0
4,"Afghanistan, Islamic Rep. of",Storm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,Zambia,TOTAL,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,2.0,3.0,1.0,1.0
966,Zimbabwe,Drought,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
967,Zimbabwe,Flood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
968,Zimbabwe,Storm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,1.0,1.0,NaN,1.0,NaN,1.0,1.0
